In [17]:
import matplotlib.pyplot as plt

import torchvision as tv
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch.optim as optim
import os
import time
import loadData8
import itertools
from sklearn.metrics import confusion_matrix
import numpy as np

In [18]:
device = torch.device("cuda:1")

In [19]:
transforms = tv.transforms.Compose([
    # transforms.CenterCrop(64),
    # transforms.CenterCrop(128),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [20]:
class Net8(nn.Module):
    def __init__(self):
        super(Net8, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(20304, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net8 = Net8()
path = './model_10dB_8个0.pkl'
net8.load_state_dict(torch.load(path))
net8 = net8.to(device)

In [21]:
class Net0(nn.Module):
    def __init__(self):
        super(Net0, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(2384, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net0 = Net0()
path = './model_10dB_all.pkl'
net0.load_state_dict(torch.load(path))
net0 = net0.to(device)

In [22]:
class Net4(nn.Module):
    def __init__(self):
        super(Net4, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(10064, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

net4 = Net4()
path = './model_10dB_4个0.pkl'
net4.load_state_dict(torch.load(path))
net4 = net4.to(device)

In [23]:
predict1, predict2, predict3 = [],[],[]
originLabels = []

In [24]:
import deleteFolder
for line in open("files_test10.txt"):   
    print(line)
    lines = line.split()
    locate, label = lines[0], lines[1]
    originData = np.load(locate)
    
    #测试8个0的情况
    with open("./split-no.txt" ,"w") as f8:
        f8.write(line)
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split-no.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)

    for data in test_loader:
        inputs, labels = data
        originLabel = int(labels)
        originLabels.append(originLabel)

        a = inputs[0:160]
#         print("---", a.shape)
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net8(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predict1.append(int(predicted[0]))
#         print(labels,"---", predicted)
    
    
    #测试输入1个0的情况
    splitData = np.split(originData,8,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/10dB/输入1个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
        
    with open("./split8.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/10dB/输入1个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
            
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split8.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net0(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
#     predict2.append(np.argmax(np.bincount(predictedLabels)))
    predict2.append([predictedLabel for predictedLabel in predictedLabels])
        
    #测试输入4个0的情况
    originData = np.load(locate)
    splitData = np.split(originData,2,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/10dB/输入4个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
    
    with open("./split2.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/10dB/输入4个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split2.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net4(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
    predict3.append([p for p in predictedLabels]) 
   
    
    print(originLabels[-1],  predict2[-1], predict3[-1], predict1[-1])

/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_115.npy 9



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


9 [10, 28, 26, 8, 9, 38, 10, 25] [10, 10] 10
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_116.npy 9



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


9 [33, 36, 9, 27, 10, 9, 28, 33] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_117.npy 9

9 [25, 33, 27, 0, 10, 33, 10, 6] [9, 25] 10
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_118.npy 9

9 [10, 9, 33, 10, 10, 1, 26, 25] [10, 8] 10
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_119.npy 9

9 [10, 33, 25, 0, 25, 33, 9, 8] [0, 9] 10
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_120.npy 9

9 [26, 9, 9, 28, 10, 1, 1, 33] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_145.npy 9

9 [9, 26, 10, 9, 9, 9, 26, 36] [1, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_146.npy 9

9 [9, 26, 10, 33, 25, 4, 25, 9] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_147.npy 9

9 [9, 25, 26, 10, 28, 9, 9, 8] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_148.npy 9

9 [9, 8, 38, 38, 25, 26, 1, 10] [9, 8] 8
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_149.npy 9

9 [38, 9, 25, 11, 6, 11, 9, 6] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处

In [25]:
# originLabels

In [26]:
# predict1

In [27]:
# predict2

In [28]:
# predict3

In [29]:
print(len(originLabels), len(predict1), len(predict2), len(predict3))

1620 1620 1620 1620


In [30]:
rightNum = 0
for i in range(len(originLabels)):
    predictAll = []
    predictAll.append(predict1[i])
    for a in predict2[i]:
        predictAll.append(a)
    for a in predict3[i]:
        predictAll.append(a)
#     print("-----",predictAll)
#     print(p.shape)
    result = np.argmax(np.bincount(predictAll))
#     print(result, originLabels[i])
    if originLabels[i] == result:
        rightNum += 1
    else:
        print(originLabels[i], predict1[i], predict2[i], predict3[i])
print(rightNum/ len(originLabels))

9 10 [10, 28, 26, 8, 9, 38, 10, 25] [10, 10]
9 10 [25, 33, 27, 0, 10, 33, 10, 6] [9, 25]
9 10 [10, 9, 33, 10, 10, 1, 26, 25] [10, 8]
9 10 [10, 33, 25, 0, 25, 33, 9, 8] [0, 9]
9 8 [9, 8, 38, 38, 25, 26, 1, 10] [9, 8]
9 9 [1, 1, 1, 33, 0, 27, 1, 25] [33, 1]
9 9 [25, 25, 25, 0, 25, 1, 1, 9] [9, 1]
9 9 [0, 33, 10, 6, 5, 0, 0, 1] [1, 9]
9 33 [9, 1, 25, 10, 33, 33, 1, 10] [33, 38]
9 26 [10, 0, 3, 25, 1, 1, 1, 25] [9, 1]
9 9 [1, 27, 0, 25, 33, 38, 27, 1] [10, 38]
9 8 [26, 9, 25, 28, 10, 25, 10, 25] [9, 25]
9 38 [25, 34, 27, 10, 1, 8, 10, 26] [10, 1]
9 10 [0, 38, 26, 8, 9, 26, 1, 9] [33, 1]
9 10 [9, 9, 38, 10, 38, 10, 27, 25] [26, 10]
9 10 [28, 31, 10, 8, 25, 7, 26, 0] [10, 9]
10 28 [25, 32, 8, 10, 3, 9, 9, 5] [28, 10]
10 8 [9, 5, 9, 10, 9, 6, 10, 1] [9, 10]
10 33 [26, 8, 9, 9, 27, 27, 10, 7] [8, 9]
10 10 [8, 9, 8, 9, 10, 8, 8, 9] [10, 10]
10 10 [31, 1, 10, 27, 1, 1, 0, 10] [10, 1]
10 10 [3, 31, 0, 0, 0, 25, 33, 0] [10, 0]
10 33 [10, 0, 10, 0, 1, 0, 0, 1] [10, 10]
11 27 [27, 0, 39, 27, 33, 39,

In [33]:
# predict2[-1] --> 1个0 , predict3[-1] --> 4个0, predict1[-1] --> 8个0
rightNum = 0
for i in range(len(originLabels)):
    predictAll = []   
    
    for _ in range(8): #单独0的情况，权重17
        for a in predict2[i]:
            predictAll.append(a)
    
    for _ in range(13): # 4个0的情况，权重13
        for a in predict3[i]:
            predictAll.append(a)
#     print("-----",predictAll)
#     print(p.shape)
    for _ in range(15):  # 8个0的情况，权重15
        predictAll.append(predict1[i])

    result = np.argmax(np.bincount(predictAll))
#     print(result, originLabels[i])
    if originLabels[i] == result:
        rightNum += 1
#     else:
#         print(originLabels[i], predict1[i], predict2[i], predict3[i])
print(rightNum/ len(originLabels))

0.7462962962962963


In [32]:
rightNum = 0

for i in range(len(originLabels)):
    if originLabels[i] == predict1[i]:
        rightNum += 1
        
print(rightNum/ len(originLabels))

0.7814814814814814
